# Eat Safe, Love

## Notebook Set Up

In [1]:
# Import dependencies
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [4]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [5]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [ ]:
# review the collections in our database
print(db.list_collection_names())

In [ ]:
# assign the collection to a variable
establishments = db.establishments
establishments.find_one()

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [ ]:
# Find the establishments with a hygiene score of 20
query = {'scores.hygiene': 20}

# Use count_documents to display the number of documents in the result
print(f"There are {establishments.count_documents(query)} establishments with a hygiene score of 20","\n","-"*40)
# Display the first document in the results using pprint
pprint(establishments.find_one(query))

In [ ]:
# Convert the result to a Pandas DataFrame
query = {'scores.Hygiene': 20}
results = establishments.find(query)
results_df = pd.DataFrame(results)

# Display the number of rows in the DataFrame
print.len(results_df)
# Display the first 10 rows of the DataFrame
results_df.head(10)

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [ ]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {'$and': [{'localAuthorityName': 'London'}, {'ratingValue': {'$gte': 4}}]
results_london = establishments.find(query)
# Use count_documents to display the number of documents in the result
print(establishments.count_documents(query))
# Display the first document in the results using pprint
pprint(results_london[0])

In [ ]:
# Convert the result to a Pandas DataFrame
query = {'LocalAuthorityName':{'$regex':'London'},'RatingValue':{'$gte':4}}
results_london = establishments.find(query)
results_london_df = pd.DataFrame(results_london)

# Display the number of rows in the DataFrame
print(len(results_london_df))
# Display the first 10 rows of the DataFrame
results_london_df.head(10)

### 3. What are the top 5 establishments with a `RatingValue` rating value of '5', sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [ ]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

query = {BusinessName: "Penang Flavours"}
fields = {'lattitude':1, 'longitude':1, '_id':0}

degree_search = 0.01
latitude = spatial_list[1]
longitude = spatial_list[0]

query = {
    'longitude': {'$gte': longitude - degree_search, '$lte': longitude + degree_search},
    'latitude': {'$gte': latitude - degree_search, '$lte': latitude + degree_search},
    'ratingValue': 5
} 
sort = [('scores.hygiene', 1)]

# Print the results
output = establishments.find(query, fields).sort(sort).limits(5)
for i in output:
    print(i)

In [ ]:
# Convert result to Pandas DataFrame
query = {
    'longitude': {'$gte': longitude - degree_search, '$lte': longitude + degree_search},
    'latitude': {'$gte': latitude - degree_search, '$lte': latitude + degree_search},
    'ratingValue': 5
} 
sort = [('scores.hygiene', 1)]

output = establishments.find(query, fields).sort(sort).limits(5)
restaurants_distance_df = pd.DataFrame(output)
restaurants_distance_df.head()

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [ ]:
# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0
match_establishments = {'scores.hygiene': 0}
# 2. Groups the matches by Local Authority
group_establishments = {'$group': {'_id': '$localAuthorityName', 'count': {'$sum': 1}}}
# 3. Sorts the matches from highest to lowest
sort_establishments = {'$sort': {'count': -1}}
pipeline = [match_establishments, group_establishments, sort_establishments]
results = establishments.aggregate(pipeline)
# Print the number of documents in the result
print.len(results)
# Print the first 10 results
pprint(results[0:10])

In [ ]:
# Convert the result to a Pandas DataFrame
poor_hygiene_establishments_df = pd.DataFrame(results)
# Display the number of rows in the DataFrame
print.len(poor_hygiene_establishments_df)
# Display the first 10 rows of the DataFrame
poor_hygiene_establishments_df.head(10)